# Binary Star Fitting Tutorial: Curve Fitting vs Bayesian Analysis

This tutorial demonstrates the challenges of fitting incomplete astronomical data using binary star radial velocity measurements as an example. We'll explore:

1. **Theoretical Background**: Understanding binary star systems and radial velocity variations
2. **Data Generation**: Creating synthetic observations with known parameters
3. **Incomplete Data Challenge**: How missing observations can lead to degeneracies
4. **Curve Fitting**: Using `scipy.optimize.curve_fit` with different initial guesses
5. **Bayesian Analysis**: Using `emcee` to explore the full parameter space

## Learning Objectives
- Understand the concept of likelihood maximization in data fitting
- Recognize challenges with incomplete astronomical data
- Identify degeneracies and multiple solutions in parameter space
- Compare frequentist vs Bayesian approaches to parameter estimation
- Understand the importance of priors and initial guesses


## 1. Theoretical Background: Binary Star Systems

Binary star systems consist of two stars orbiting their common center of mass. When we observe one star (the primary), its radial velocity (line-of-sight velocity) varies periodically due to the gravitational influence of its companion.

### Key Parameters:
- **Period (P)**: Orbital period in days
- **Eccentricity (e)**: Shape of the orbit (between 0 and 1, 0 = circular, 1 = parabolic)
- **Argument of periastron (ω)**: Orientation of the orbit
- **Time of periastron passage (T₀)**: When the star is closest to the companion
- **Velocity amplitude (K)**: Maximum radial velocity variation
- **Systemic velocity (γ)**: Average velocity of the system

### Radial Velocity Equation:
The radial velocity of the primary star is given by:

$$v_r(t) = \gamma + K[\cos(\omega + f(t)) + e\cos(\omega)]$$

where $f(t)$ is the true anomaly, which depends on the orbital parameters and time.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import emcee
import corner
from astropy import units as u
from astropy.constants import G
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Set random seed for reproducibility
np.random.seed(42)


## 2. Radial Velocity Model Implementation

Let's implement the radial velocity model for a binary star system:


In [ ]:
def solve_kepler_equation(E, e):
    """Solve Kepler's equation: M = E - e*sin(E)"""
    M = E - e * np.sin(E)
    return M

def get_true_anomaly(E, e):
    """Calculate true anomaly from eccentric anomaly"""
    f = 2 * np.arctan(np.sqrt((1 + e) / (1 - e)) * np.tan(E / 2))
    return f

def radial_velocity_model(t, P, e, omega, T0, K, gamma):
    """
    Calculate radial velocity for a binary star system
    
    Parameters:
    -----------
    t : array_like
        Time array
    P : float
        Orbital period in days
    e : float
        Eccentricity (0-1)
    omega : float
        Argument of periastron in radians
    T0 : float
        Time of periastron passage
    K : float
        Velocity amplitude in km/s
    gamma : float
        Systemic velocity in km/s
    
    Returns:
    --------
    v_r : array_like
        Radial velocity in km/s
    """
    # Mean anomaly
    M = 2 * np.pi * (t - T0) / P
    
    # Solve for eccentric anomaly using Newton-Raphson
    E = M.copy()  # Initial guess
    for _ in range(10):  # Max iterations
        E_new = E - (E - e * np.sin(E) - M) / (1 - e * np.cos(E))
        if np.allclose(E, E_new, rtol=1e-10):
            break
        E = E_new
    
    # True anomaly
    f = get_true_anomaly(E, e)
    
    # Radial velocity
    v_r = gamma + K * (np.cos(omega + f) + e * np.cos(omega))
    
    return v_r

# Test the model with some example parameters
t_test = np.linspace(0, 500, 50)
P_test, e_test, omega_test, T0_test, K_test, gamma_test = 150.0, 0.3, np.pi/3, 30.0, 20.0, 50.0

v_r_test = radial_velocity_model(t_test, P_test, e_test, omega_test, T0_test, K_test, gamma_test)

In [ ]:
# Plot the results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(t_test, v_r_test, c='b', ls='None', marker='o', linewidth=2)
plt.xlabel('Time (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Example Binary Star Radial Velocity Curve')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(t_test % P_test, v_r_test, c='b', ls='None', marker='o', linewidth=2)
plt.xlabel('Phase')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Folded Light Curve')
plt.grid(True, alpha=0.3)
# plt.tight_layout()
plt.show()

print(f"Model parameters:")
print(f"Period: {P_test} days")
print(f"Eccentricity: {e_test}")
print(f"Omega: {omega_test:.3f} rad ({np.degrees(omega_test):.1f}°)")
print(f"T0: {T0_test} days")
print(f"K: {K_test} km/s")
print(f"Gamma: {gamma_test} km/s")


## 3. Generate Synthetic Data with Known Parameters

Now let's create synthetic observational data. We'll use parameters that create a challenging fitting scenario:


In [ ]:
# True parameters for our synthetic binary system
true_params = {
    'P': 150.0,      # Period in days
    'e': 0.3,       # Eccentricity
    'omega': np.pi/3,  # Argument of periastron
    'T0': 30,     # Time of periastron passage
    'K': 20.0,      # Velocity amplitude in km/s
    'gamma': 50.0   # Systemic velocity in km/s
}

print("True parameters:")
for key, value in true_params.items():
    if key == 'omega':
        print(f"{key}: {value:.3f} rad ({np.degrees(value):.1f}°)")
    else:
        print(f"{key}: {value}")

# Generate full time series (3 orbital periods)
t_full = np.linspace(0, 3 * true_params['P'], 50)
v_r_true = radial_velocity_model(t_full, **true_params)

# Add realistic measurement uncertainties
sigma_obs = 3.0  # km/s
v_r_obs_full = v_r_true + np.random.normal(0, sigma_obs, len(t_full))

# Generate true model line
t_model = np.linspace(0, 3 * true_params['P'], 1000)
v_r_model = radial_velocity_model(t_model, **true_params)


In [ ]:
# Plot the full dataset
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(t_model, v_r_model, 'b-', linewidth=2, label='True model', alpha=0.7)
plt.errorbar(t_full, v_r_obs_full, yerr=sigma_obs, fmt='ko', markersize=4, 
             alpha=0.6, label='Observations')
plt.xlabel('Time (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Full Synthetic Dataset')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
phase_full = (t_full - true_params['T0']) % true_params['P']
phase_model = (t_model - true_params['T0']) % true_params['P']
# only plot one phase of the model
mask_phase1 = ((t_model - true_params['T0']) // true_params['P']) == 0
plt.plot(phase_model[mask_phase1], v_r_model[mask_phase1], 'b-', linewidth=2, label='True model', alpha=0.7)
plt.errorbar(phase_full, v_r_obs_full, yerr=sigma_obs, fmt='ko', markersize=4, 
             alpha=0.6, label='Observations')
plt.xlabel('Phase (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Folded Light Curve')
plt.legend()
plt.grid(True, alpha=0.3)

# plt.tight_layout()
plt.show()

print(f"\nGenerated {len(t_full)} data points over {3 * true_params['P']:.1f} days")
print(f"Measurement uncertainty: {sigma_obs} km/s")

## 4. Create Incomplete Data Scenario

Now comes the crucial part: we'll strategically remove observations to create a scenario where multiple solutions are equally plausible. This simulates real-world situations where:
- Weather prevents observations
- Telescope time is limited
- Seasonal constraints exist
- Equipment failures occur

We'll remove data points in a way that creates degeneracies in the parameter space.


In [ ]:
# Create incomplete dataset by strategically removing observations
# This creates a scenario where multiple solutions are equally valid

# Keep only observations in specific phase ranges to create degeneracies
phase_full = (t_full - true_params['T0']) % true_params['P']

# Define phase ranges to keep (this creates the degeneracy)
keep_phases = [
    (0.05, 0.08),      # Near periastron
    (0.47, 0.51),    # Mid-orbit
    (0.31, 0.33),    # Mid-orbit
    (0.83, 0.95)     # Near apastron
]

# Create mask for observations to keep
keep_mask = np.zeros(len(t_full), dtype=bool)
for phase_range in keep_phases:
    phase_norm = phase_full / true_params['P']  # Normalize to 0-1
    mask = (phase_norm >= phase_range[0]) & (phase_norm <= phase_range[1])
    keep_mask |= mask

# Apply the mask
t_obs = t_full[keep_mask]
v_r_obs = v_r_obs_full[keep_mask]
sigma_obs_array = np.full(len(t_obs), sigma_obs)

# Plot comparison
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(t_model, v_r_model, 'b-', linewidth=2, label='True model', alpha=0.7)
plt.errorbar(t_full, v_r_obs_full, yerr=sigma_obs, fmt='ko', markersize=3, 
             alpha=0.4, label='All observations')
plt.xlabel('Time (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Full Dataset')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(t_model, v_r_model, 'b-', linewidth=2, label='True model', alpha=0.7)
plt.errorbar(t_obs, v_r_obs, yerr=sigma_obs_array, fmt='ro', markersize=5, 
             label='Kept observations')
plt.xlabel('Time (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Incomplete Dataset')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
phase_obs = (t_obs - true_params['T0']) % true_params['P']
plt.plot(phase_model[mask_phase1], v_r_model[mask_phase1], 'b-', linewidth=2, label='True model', alpha=0.7)
plt.errorbar(phase_obs, v_r_obs, yerr=sigma_obs_array, fmt='ro', markersize=5, 
             label='Kept observations')
plt.xlabel('Phase (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Folded Incomplete Data')
plt.legend()
plt.grid(True, alpha=0.3)

# plt.tight_layout()
plt.show()

print(f"\nData reduction:")
print(f"Full dataset: {len(t_full)} points")
print(f"Incomplete dataset: {len(t_obs)} points")
print(f"Data coverage: {len(t_obs)/len(t_full)*100:.1f}%")
print(f"\nThis incomplete dataset will lead to degeneracies in parameter space!")


In [ ]:
# Export the complete and incomplete data as csv files
import pandas as pd
df_out = pd.DataFrame({
    'time':t_full, 
    'vlos':v_r_obs_full
})
df_out.to_csv("binary_measurements_complete.csv",index=False)
df_out = pd.DataFrame({
    'time':t_obs, 
    'vlos':v_r_obs,
    'error':sigma_obs_array
})
df_out.to_csv("binary_measurements_observed.csv",index=False)

## 5. Curve Fitting with Good Initial Guess

Let's start with `curve_fit` using initial guesses close to the true parameters. This should work well and recover the true solution.


In [ ]:
# Define the function for curve_fit (only takes time and parameters)
def rv_model_for_fit(t, P, e, omega, T0, K, gamma):
    return radial_velocity_model(t, P, e, omega, T0, K, gamma)

# Good initial guess (close to true parameters)
p0_good = [
    true_params['P'] * 1.05,      # Period: 5% off
    true_params['e'] * 0.9,       # Eccentricity: 10% off
    true_params['omega'] + 0.1,   # Omega: small offset
    true_params['T0'] + 2.0,      # T0: 2 days off
    true_params['K'] * 1.1,       # K: 10% off
    true_params['gamma'] + 1.0    # Gamma: 1 km/s off
]

# Parameter bounds for curve_fit
bounds = (
    [10, 0, 0, 0, 5, -100],      # Lower bounds
    [200, 0.99, 2*np.pi, 50, 50, 100]  # Upper bounds
)

print("Good initial guess:")
param_names = ['P', 'e', 'omega', 'T0', 'K', 'gamma']
for i, name in enumerate(param_names):
    if name == 'omega':
        print(f"{name}: {p0_good[i]:.3f} rad ({np.degrees(p0_good[i]):.1f}°) - True: {true_params[name]:.3f} rad ({np.degrees(true_params[name]):.1f}°)")
    else:
        print(f"{name}: {p0_good[i]:.3f} - True: {true_params[name]:.3f}")

# Perform the fit
popt_good, pcov_good = curve_fit(
    rv_model_for_fit, t_obs, v_r_obs, 
    p0=p0_good, sigma=sigma_obs_array, 
    bounds=bounds, maxfev=10000
)

# Calculate uncertainties
perr_good = np.sqrt(np.diag(pcov_good))

print("\nFit results (good initial guess):")
print("Parameter | Best Fit | Uncertainty | True Value | Difference")
print("-" * 60)

for i, name in enumerate(param_names):
    true_val = true_params[name]
    fit_val = popt_good[i]
    err_val = perr_good[i]
    diff = fit_val - true_val
    
    if name == 'omega':
        print(f"{name:8s} | {fit_val:8.3f} | {err_val:8.3f} | {true_val:8.3f} | {diff:8.3f}")
    else:
        print(f"{name:8s} | {fit_val:8.3f} | {err_val:8.3f} | {true_val:8.3f} | {diff:8.3f}")

# Calculate chi-squared
v_r_fit_good = rv_model_for_fit(t_obs, *popt_good)
chi2_good = np.sum(((v_r_obs - v_r_fit_good) / sigma_obs_array)**2)
dof_good = len(t_obs) - len(popt_good)
reduced_chi2_good = chi2_good / dof_good

print(f"\nChi-squared: {chi2_good:.2f}")
print(f"Degrees of freedom: {dof_good}")
print(f"Reduced chi-squared: {reduced_chi2_good:.2f}")


In [ ]:
# Plot the results
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
t_plot = np.linspace(0, 3*true_params['P'], 1000)
v_r_true_plot = radial_velocity_model(t_plot, **true_params)
v_r_fit_plot = rv_model_for_fit(t_plot, *popt_good)

plt.plot(t_model, v_r_model, 'b-', linewidth=2, label='True model', alpha=0.7)
plt.plot(t_plot, v_r_fit_plot, 'r--', linewidth=2, label='Best fit')
plt.errorbar(t_obs, v_r_obs, yerr=sigma_obs_array, fmt='ko', markersize=5, 
                alpha=0.7, label='Data')
plt.xlabel('Time (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Good Initial Guess Fit')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
phase_plot = (t_plot - popt_good[3]) % popt_good[0]
mask_bestfit_phase1 = ((t_plot - popt_good[3]) // popt_good[0]) == 0
phase_obs = (t_obs - popt_good[3]) % popt_good[0]

plt.plot(phase_model[mask_phase1], v_r_model[mask_phase1], 'b-', linewidth=2, label='True model', alpha=0.7)
plt.plot(phase_plot[mask_bestfit_phase1], v_r_fit_plot[mask_bestfit_phase1], 'g--', linewidth=2, label='Best fit')
plt.errorbar(phase_obs, v_r_obs, yerr=sigma_obs_array, fmt='ko', markersize=5, 
                alpha=0.7, label='Data')
plt.xlabel('Phase (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Folded Good Fit')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
residuals = v_r_obs - v_r_fit_good
plt.errorbar(t_obs, residuals, yerr=sigma_obs_array, fmt='go', markersize=5)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.7)
plt.xlabel('Time (days)')
plt.ylabel('Residuals (km/s)')
plt.title('Residuals')
plt.grid(True, alpha=0.3)

# plt.tight_layout()
plt.show()

## 6. Curve Fitting with Bad Initial Guess

Now let's try the same fitting procedure with a poor initial guess. This will demonstrate how the choice of starting parameters can lead to completely different (but equally valid) solutions.


In [ ]:
# Bad initial guess (far from true parameters)
p0_bad = [
    350.0,           # Period: much longer
    0.6,            # Eccentricity: much larger
    np.pi,          # Omega: 120 degrees different
    50.0,           # T0: much later
    20.0,           # K: This is usually hard to get wrong
    50.0            # Gamma: This is usually hard to get wrong
]

# Parameter bounds for curve_fit
bounds = (
    [10, 0, 0, 0, 5, -100],      # Lower bounds
    [500, 0.99, 2*np.pi, 100, 50, 100]  # Upper bounds
)

print("Bad initial guess:")
for i, name in enumerate(param_names):
    if name == 'omega':
        print(f"{name}: {p0_bad[i]:.3f} rad ({np.degrees(p0_bad[i]):.1f}°) - True: {true_params[name]:.3f} rad ({np.degrees(true_params[name]):.1f}°)")
    else:
        print(f"{name}: {p0_bad[i]:.3f} - True: {true_params[name]:.3f}")

# Perform the fit with bad initial guess
popt_bad, pcov_bad = curve_fit(
    rv_model_for_fit, t_obs, v_r_obs, 
    p0=p0_bad, sigma=sigma_obs_array, 
    bounds=bounds, maxfev=10000
)

# Calculate uncertainties
perr_bad = np.sqrt(np.diag(pcov_bad))

print("\nFit results (bad initial guess):")
print("Parameter | Best Fit | Uncertainty | True Value | Difference")
print("-" * 60)

for i, name in enumerate(param_names):
    true_val = true_params[name]
    fit_val = popt_bad[i]
    err_val = perr_bad[i]
    diff = fit_val - true_val
    
    if name == 'omega':
        print(f"{name:8s} | {fit_val:8.3f} | {err_val:8.3f} | {true_val:8.3f} | {diff:8.3f}")
    else:
        print(f"{name:8s} | {fit_val:8.3f} | {err_val:8.3f} | {true_val:8.3f} | {diff:8.3f}")

# Calculate chi-squared
v_r_fit_bad = rv_model_for_fit(t_obs, *popt_bad)
chi2_bad = np.sum(((v_r_obs - v_r_fit_bad) / sigma_obs_array)**2)
dof_bad = len(t_obs) - len(popt_bad)
reduced_chi2_bad = chi2_bad / dof_bad

print(f"\nChi-squared: {chi2_bad:.2f}")
print(f"Degrees of freedom: {dof_bad}")
print(f"Reduced chi-squared: {reduced_chi2_bad:.2f}")




In [ ]:
# Compare the two fits
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
t_plot = np.linspace(0, 3*true_params['P'], 1000)
v_r_true_plot = radial_velocity_model(t_plot, **true_params)
v_r_fit_good_plot = rv_model_for_fit(t_plot, *popt_good)
v_r_fit_bad_plot = rv_model_for_fit(t_plot, *popt_bad)

plt.plot(t_model, v_r_model, 'b-', linewidth=2, label='True model', alpha=0.7)
plt.plot(t_plot, v_r_fit_good_plot, 'g--', linewidth=2, label='Good guess fit')
plt.plot(t_plot, v_r_fit_bad_plot, 'r--', linewidth=2, label='Bad guess fit')
plt.errorbar(t_obs, v_r_obs, yerr=sigma_obs_array, fmt='ko', markersize=5, 
                alpha=0.7, label='Data')
plt.xlabel('Time (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Comparison of Fits')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
phase_plot_bad = (t_plot - popt_bad[3]) % popt_bad[0]
mask_badfit_phase1 = ((t_plot - popt_bad[3]) // popt_bad[0]) == 0
phase_obs_bad = (t_obs - popt_bad[3]) % popt_bad[0]

plt.plot(phase_model[mask_phase1], v_r_model[mask_phase1], 'b-', linewidth=2, label='True model', alpha=0.7)
plt.plot(phase_plot[mask_bestfit_phase1], v_r_fit_plot[mask_bestfit_phase1], 'g--', linewidth=2, label='Good guess fit')
plt.plot(phase_plot_bad[mask_badfit_phase1], v_r_fit_bad_plot[mask_badfit_phase1], 'r--', linewidth=2, label='Bad guess fit')
plt.errorbar(phase_obs, v_r_obs, yerr=sigma_obs_array, fmt='go', markersize=5, 
                alpha=0.7, label='Data')
plt.errorbar(phase_obs_bad, v_r_obs, yerr=sigma_obs_array, fmt='ro', markersize=5, 
                alpha=0.7, label='Data')
plt.xlabel('Phase (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Folded Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
residuals_good = v_r_obs - v_r_fit_good
residuals_bad = v_r_obs - v_r_fit_bad

plt.errorbar(t_obs, residuals_good, yerr=sigma_obs_array, fmt='go', markersize=5, 
                alpha=0.7, label='Good guess')
plt.errorbar(t_obs, residuals_bad, yerr=sigma_obs_array, fmt='ro', markersize=5, 
                alpha=0.7, label='Bad guess')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.7)
plt.xlabel('Time (days)')
plt.ylabel('Residuals (km/s)')
plt.title('Residuals Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# plt.tight_layout()
plt.show()

print(f"\nKey Insight: Both fits have similar chi-squared values!")
print(f"Good guess chi-squared: {chi2_good:.2f}")
print(f"Bad guess chi-squared: {chi2_bad:.2f}")
print(f"Difference: {abs(chi2_good - chi2_bad):.2f}")
print(f"\nThis demonstrates the degeneracy problem with incomplete data!")

## 7. Bayesian Analysis with emcee

Now let's use `emcee` to perform a Bayesian analysis. This will explore the entire parameter space and reveal all possible solutions, not just the local minimum found by `curve_fit`.


In [ ]:
# Define the log-likelihood function
def log_likelihood(theta, t, v_r, sigma):
    """
    Log-likelihood function for the radial velocity model
    """
    P, e, omega, T0, K, gamma = theta
    
    # Calculate model predictions
    v_r_model = radial_velocity_model(t, P, e, omega, T0, K, gamma)
    
    # Calculate log-likelihood (assuming Gaussian errors)
    log_like = -0.5 * np.sum(((v_r - v_r_model) / sigma)**2 + np.log(2 * np.pi * sigma**2))
    
    return log_like

# Define the log-prior function
def log_prior(theta):
    """
    Log-prior function with broad, uninformative priors
    """
    P, e, omega, T0, K, gamma = theta
    
    # Broad priors
    if (10 < P < 500 and 
        0 < e < 0.99 and 
        0 < omega < 2*np.pi and 
        0 < T0 < 100 and 
        5 < K < 50 and 
        -100 < gamma < 100):
        return 0.0  # Uniform prior
    
    return -np.inf  # Outside prior range


# Define the log-posterior function
def log_posterior(theta, t, v_r, sigma):
    """
    Log-posterior function
    """
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    
    return lp + log_likelihood(theta, t, v_r, sigma)

# Set up the MCMC sampler
nwalkers = 50  # Number of walkers
ndim = 6       # Number of parameters
nsteps = 2000  # Number of steps

# Initial positions for walkers (scattered around the parameter space)
initial_positions = np.array([
    np.random.uniform(20, 490, nwalkers),      # P
    np.random.uniform(0.1, 0.8, nwalkers),     # e
    np.random.uniform(0, 2*np.pi, nwalkers),   # omega
    np.random.uniform(0, 80, nwalkers),        # T0
    np.random.uniform(10, 40, nwalkers),       # K
    np.random.uniform(-90, 90, nwalkers)       # gamma
]).T

print(f"Setting up MCMC with {nwalkers} walkers, {nsteps} steps")
print(f"Initial positions spread across parameter space")

# Create the sampler
sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_posterior, 
    args=(t_obs, v_r_obs, sigma_obs_array)
)

# Run the MCMC
print("Running MCMC...")
sampler.run_mcmc(initial_positions, nsteps, progress=True)

# Get the samples (discard burn-in)
burn_in = 500
samples = sampler.get_chain(discard=burn_in, flat=True)

print(f"\nMCMC completed!")
print(f"Discarded {burn_in} steps as burn-in")
print(f"Final sample size: {len(samples)} points")

# Calculate acceptance fraction
acceptance_fraction = np.mean(sampler.acceptance_fraction)
print(f"Average acceptance fraction: {acceptance_fraction:.3f}")

# Plot the chain evolution
fig, axes = plt.subplots(ndim, 1, figsize=(12, 10), sharex=True)
for i, param_name in enumerate(param_names):
    axes[i].plot(sampler.get_chain()[:, :, i], alpha=0.3)
    axes[i].axvline(burn_in, color='r', linestyle='--', alpha=0.7)
    axes[i].set_ylabel(param_name)
    if i == ndim - 1:
        axes[i].set_xlabel('Step')

plt.suptitle('MCMC Chain Evolution')
plt.tight_layout()
plt.show()


## 8. Analyze the Posterior Distribution

Now let's analyze the posterior distribution to see what the Bayesian analysis reveals about the parameter space.


In [ ]:
# Calculate summary statistics
param_medians = np.median(samples, axis=0)
param_errors = np.std(samples, axis=0)
param_16th = np.percentile(samples, 16, axis=0)
param_84th = np.percentile(samples, 84, axis=0)

print("Bayesian Analysis Results:")
print("Parameter | Median | 16th %ile | 84th %ile | True Value | In Range?")
print("-" * 70)

for i, name in enumerate(param_names):
    true_val = true_params[name]
    median_val = param_medians[i]
    low_val = param_16th[i]
    high_val = param_84th[i]
    
    # Check if true value is within 1-sigma range
    in_range = low_val <= true_val <= high_val
    
    if name == 'omega':
        print(f"{name:8s} | {median_val:6.3f} | {low_val:8.3f} | {high_val:8.3f} | {true_val:8.3f} | {str(in_range):8s}")
    else:
        print(f"{name:8s} | {median_val:6.3f} | {low_val:8.3f} | {high_val:8.3f} | {true_val:8.3f} | {str(in_range):8s}")

# Create corner plot
fig = corner.corner(
    samples, 
    labels=param_names,
    truths=[true_params[name] for name in param_names],
    quantiles=[0.16, 0.5, 0.84],
    show_titles=True,
    title_kwargs={"fontsize": 12}
)

plt.suptitle('Posterior Distribution (Bayesian Analysis)', fontsize=16)
# plt.tight_layout()
plt.show()

# Plot individual parameter distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, name in enumerate(param_names):
    ax = axes[i]
    
    # Histogram of samples
    ax.hist(samples[:, i], bins=50, alpha=0.7, density=True, color='skyblue')
    
    # True value
    ax.axvline(true_params[name], color='red', linestyle='--', linewidth=2, 
               label='True value')
    
    # Median and confidence intervals
    ax.axvline(param_medians[i], color='black', linestyle='-', linewidth=2, 
               label='Median')
    ax.axvline(param_16th[i], color='black', linestyle=':', alpha=0.7)
    ax.axvline(param_84th[i], color='black', linestyle=':', alpha=0.7)
    
    # Curve fit results (if available)
    if popt_good is not None:
        ax.axvline(popt_good[i], color='green', linestyle='--', linewidth=2, 
                   label='Curve fit (good)')
    if popt_bad is not None:
        ax.axvline(popt_bad[i], color='orange', linestyle='--', linewidth=2, 
                   label='Curve fit (bad)')
    
    ax.set_xlabel(name)
    ax.set_ylabel('Density')
    ax.set_title(f'{name} Posterior Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)

# plt.tight_layout()
plt.show()

# Calculate the number of distinct solutions
print("\nAnalyzing posterior for multiple solutions...")

# Look for bimodal distributions (especially in omega)
omega_samples = samples[:, 2]  # omega parameter
omega_hist, omega_bins = np.histogram(omega_samples, bins=100)
omega_centers = (omega_bins[:-1] + omega_bins[1:]) / 2

# Find peaks in the omega distribution
from scipy.signal import find_peaks
peaks, _ = find_peaks(omega_hist, height=np.max(omega_hist)*0.1, distance=10)

print(f"Found {len(peaks)} peaks in omega distribution")
for i, peak in enumerate(peaks):
    peak_omega = omega_centers[peak]
    print(f"Peak {i+1}: omega = {peak_omega:.3f} rad ({np.degrees(peak_omega):.1f}°)")

print(f"\nTrue omega: {true_params['omega']:.3f} rad ({np.degrees(true_params['omega']):.1f}°)")


## 9. Model Comparison and Predictions

Let's compare the predictions from different approaches and see how they perform on unseen data.


In [ ]:
# Generate predictions for future times
t_future = np.linspace(3*true_params['P'], 5*true_params['P'], 1000)
v_r_true_future = radial_velocity_model(t_future, **true_params)

# Predictions from curve fit (good guess)
if popt_good is not None:
    v_r_pred_good = rv_model_for_fit(t_future, *popt_good)
else:
    v_r_pred_good = None

# Predictions from curve fit (bad guess)
if popt_bad is not None:
    v_r_pred_bad = rv_model_for_fit(t_future, *popt_bad)
else:
    v_r_pred_bad = None

# Predictions from Bayesian analysis (using median parameters)
v_r_pred_bayesian = radial_velocity_model(t_future, *param_medians)

# Calculate prediction uncertainties from Bayesian samples
v_r_pred_samples = np.array([radial_velocity_model(t_future, *sample) for sample in samples[::10]])
v_r_pred_low = np.percentile(v_r_pred_samples, 16, axis=0)
v_r_pred_high = np.percentile(v_r_pred_samples, 84, axis=0)

# Plot predictions
plt.figure(figsize=(15, 8))

plt.subplot(2, 1, 1)
plt.plot(t_future, v_r_true_future, 'b-', linewidth=3, label='True model', alpha=0.8)

if v_r_pred_good is not None:
    plt.plot(t_future, v_r_pred_good, 'g--', linewidth=2, label='Curve fit (good guess)')

if v_r_pred_bad is not None:
    plt.plot(t_future, v_r_pred_bad, 'r--', linewidth=2, label='Curve fit (bad guess)')

plt.plot(t_future, v_r_pred_bayesian, 'purple', linestyle='-', linewidth=2, label='Bayesian (median)')
plt.fill_between(t_future, v_r_pred_low, v_r_pred_high, alpha=0.3, color='purple', 
                 label='Bayesian 1σ range')

# Show the training data
plt.errorbar(t_obs, v_r_obs, yerr=sigma_obs_array, fmt='ko', markersize=6, 
             alpha=0.7, label='Training data')

plt.xlabel('Time (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Model Predictions for Future Times')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
# Focus on a smaller time range for better visibility
t_focus = t_future[t_future <= 4*true_params['P']]
v_r_true_focus = v_r_true_future[t_future <= 4*true_params['P']]

plt.plot(t_focus, v_r_true_focus, 'b-', linewidth=3, label='True model', alpha=0.8)

if v_r_pred_good is not None:
    v_r_pred_good_focus = v_r_pred_good[t_future <= 4*true_params['P']]
    plt.plot(t_focus, v_r_pred_good_focus, 'g--', linewidth=2, label='Curve fit (good guess)')

if v_r_pred_bad is not None:
    v_r_pred_bad_focus = v_r_pred_bad[t_future <= 4*true_params['P']]
    plt.plot(t_focus, v_r_pred_bad_focus, 'r--', linewidth=2, label='Curve fit (bad guess)')

v_r_pred_bayesian_focus = v_r_pred_bayesian[t_future <= 4*true_params['P']]
v_r_pred_low_focus = v_r_pred_low[t_future <= 4*true_params['P']]
v_r_pred_high_focus = v_r_pred_high[t_future <= 4*true_params['P']]

plt.plot(t_focus, v_r_pred_bayesian_focus, 'purple', linestyle='-', linewidth=2, label='Bayesian (median)')
plt.fill_between(t_focus, v_r_pred_low_focus, v_r_pred_high_focus, alpha=0.3, color='purple', 
                 label='Bayesian 1σ range')

plt.errorbar(t_obs, v_r_obs, yerr=sigma_obs_array, fmt='ko', markersize=6, 
             alpha=0.7, label='Training data')

plt.xlabel('Time (days)')
plt.ylabel('Radial Velocity (km/s)')
plt.title('Zoomed View of Predictions')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate prediction errors
print("\nPrediction Errors (RMS):")
print("-" * 30)

if v_r_pred_good is not None:
    error_good = np.sqrt(np.mean((v_r_true_future - v_r_pred_good)**2))
    print(f"Curve fit (good guess): {error_good:.2f} km/s")

if v_r_pred_bad is not None:
    error_bad = np.sqrt(np.mean((v_r_true_future - v_r_pred_bad)**2))
    print(f"Curve fit (bad guess): {error_bad:.2f} km/s")

error_bayesian = np.sqrt(np.mean((v_r_true_future - v_r_pred_bayesian)**2))
print(f"Bayesian (median): {error_bayesian:.2f} km/s")

print(f"\nKey Insights:")
print(f"1. The Bayesian approach provides uncertainty estimates")
print(f"2. Multiple solutions are possible with incomplete data")
print(f"3. The choice of initial guess significantly affects curve_fit results")
print(f"4. Bayesian analysis reveals the full parameter space")
print(f"5. Future observations will help distinguish between solutions")


## 10. Summary and Conclusions

This tutorial has demonstrated several important concepts in astronomical data analysis:

### Key Takeaways:

1. **Incomplete Data Challenges**: When observations are sparse or incomplete, multiple solutions can fit the data equally well. This is a common problem in astronomy.

2. **Initial Guess Sensitivity**: The `curve_fit` method is sensitive to initial parameter guesses. With incomplete data, different starting points can lead to different (but equally valid) solutions.

3. **Bayesian Advantages**: The `emcee` approach explores the entire parameter space and reveals all possible solutions, not just local minima.

4. **Uncertainty Quantification**: Bayesian methods provide proper uncertainty estimates that account for parameter correlations and degeneracies.

5. **Prior Information**: The choice of priors in Bayesian analysis can significantly influence results, especially with limited data.

### Real-World Applications:

- **Exoplanet Detection**: Radial velocity surveys often have incomplete phase coverage
- **Binary Star Studies**: Limited observing windows due to weather, telescope scheduling
- **Asteroid Orbits**: Sparse observations over long time periods
- **Galaxy Dynamics**: Limited spatial coverage in velocity field measurements

### Best Practices:

1. **Always check multiple initial guesses** when using optimization methods
2. **Use Bayesian methods** when dealing with incomplete or sparse data
3. **Consider parameter degeneracies** and their physical implications
4. **Plan observations** to break degeneracies when possible
5. **Report uncertainties** that reflect the true parameter space

### Further Exploration:

- Try different incomplete data scenarios
- Experiment with different prior choices
- Add more realistic noise models
- Explore parameter correlations in detail
- Implement model comparison techniques
